In [1]:
import os
import re
import io
import pprint
import hashlib
import requests
import urllib.request
from PIL import Image
from bs4 import BeautifulSoup
from mongoengine import connect, Document, fields

In [2]:
# Scraping IMDb website as it has all the celebrity names in alphabetical order.....
res = requests.get('https://www.imdb.com/list/ls068010962/')
soup = BeautifulSoup(res.text, 'html.parser')

In [3]:
# Getting images from website.
image_links = []
for img in soup.find_all('img'):
    image_links.append(img.get('src'))

pprint.pprint(image_links)

['https://m.media-amazon.com/images/M/MV5BMjAwMjk3NDUzN15BMl5BanBnXkFtZTcwNjI4MTY0NA@@._V1_UX140_CR0,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BODI4NDY1NzkyM15BMl5BanBnXkFtZTgwNzM3MDM0OTE@._V1_UY209_CR13,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMTY0MDY2MDMwOV5BMl5BanBnXkFtZTgwNDE4MDM0OTE@._V1_UY209_CR6,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BOWUzNjU3YTUtOGIyMi00MDk3LTk1NDMtMzY4MDZlNDFhNTFhXkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR73,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BNTk1OTUxMzIzMV5BMl5BanBnXkFtZTcwMzMxMjI0Nw@@._V1_UY209_CR6,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BNzZhNDQ1YzItNTVlOC00MmVmLTkzMjEtMDFmN2Q0MDU2YmQ3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR21,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMTU2NzkzODA3OV5BMl5BanBnXkFtZTcwMzk2MDMyOA@@._V1_UX140_CR0,0,140,209_AL_.jpg',
 'https://m.media-amazon.com/images/M/MV5BMjAxNDQyNDkxM15BMl5BanBnXkFtZTcwNTQyNTA0MQ@@

In [4]:
Names = []
for img in soup.find_all('img'):
    Names.append(img.get('alt'))
pprint.pprint(Names)

['Aamir Khan',
 'Akshay Kumar',
 'Ajay Devgn',
 'Amjad Khan',
 'Amitabh Bachchan',
 'Amol Palekar',
 'Abhay Deol',
 'Akshaye Khanna',
 'Arshad Warsi',
 'Anushka Shetty',
 'Atul Kulkarni',
 'Asrani',
 'Ayushmann Khurrana',
 'Aruna Irani',
 'Annu Kapoor',
 'Ashish Vidyarthi',
 'Adil Hussain',
 'Anupam Kher',
 'Anil Kapoor',
 'Amrish Puri',
 'Amole Gupte',
 'Boman Irani',
 'Balraj Sahni',
 'Binnu Dhillon',
 'Bharat Bhushan',
 'Chiranjeevi',
 'Chunky Pandey',
 'Danny Denzongpa',
 'Darsheel Safary',
 'Deepak Tijori',
 'Dharmendra',
 'Deepika Padukone',
 'Dimple Kapadia',
 'Dev Anand',
 'Deepti Naval',
 'Dilip Kumar',
 'Farooq Shaikh',
 'Farhan Akhtar',
 'Feroz Khan',
 'Farida Jalal',
 'Guru Dutt',
 'Govinda',
 'Gulshan Grover',
 'Gurpreet Ghuggi',
 'Girish Karnad',
 'Hrithik Roshan',
 'Huma Qureshi',
 'Inaamulhaq',
 'Irrfan Khan',
 'Jackie Shroff',
 'Jimmy Sheirgill',
 'Jeetendra',
 'Jaspal Bhatti',
 'Johnny Lever',
 'Jaswinder Bhalla',
 'Kamal Haasan',
 'Kangana Ranaut',
 'Konkona Sen Shar

In [5]:
infos = []
line = str()
final_info = []
infos = soup.find_all('p',class_= False)

for info in infos:
    info = info.get_text()
    info = info.rstrip('\n')
    #print(info)
    final_info.append(info)

final_info = final_info[1:]
pprint.pprint(final_info)

['\n'
 '    Aamir is no doubt one of the most dedicated actors in this world. With '
 'his recent success in India and China markets combined he has got the title '
 "of World's Biggest Superstar. He went through rapid transformations in his "
 'body structure for his films to bring out the reality factor in his '
 '...                ',
 '\n'
 '    Akshay Kumar, born September 9, 1967, is an actor. He studied martial '
 'arts in Hong Kong. It was a student that suggested that he try modeling. '
 'Because of his success as model, he was offered films. Along with his good '
 'looks and excellent martial art skills, he was always the first choice to do '
 '...                ',
 '\n'
 '    Ajay Devgn is an Indian film actor, director, and producer. He made his '
 'film debut with Phool Aur Kaante in 1991 and received a Filmfare Award for '
 'Best Male Debut for his performance in the film. He then starred in films '
 'such as Jigar (1992), Dilwale (1994), Suhaag (1994), Naajayaz (1995), 

In [6]:
bollywood_dictionary = dict()
bollywood_dictionary["Names"] = Names
bollywood_dictionary["Images"] = image_links
bollywood_dictionary["Info"] = final_info

In [7]:
pprint.pprint(bollywood_dictionary)

{'Images': ['https://m.media-amazon.com/images/M/MV5BMjAwMjk3NDUzN15BMl5BanBnXkFtZTcwNjI4MTY0NA@@._V1_UX140_CR0,0,140,209_AL_.jpg',
            'https://m.media-amazon.com/images/M/MV5BODI4NDY1NzkyM15BMl5BanBnXkFtZTgwNzM3MDM0OTE@._V1_UY209_CR13,0,140,209_AL_.jpg',
            'https://m.media-amazon.com/images/M/MV5BMTY0MDY2MDMwOV5BMl5BanBnXkFtZTgwNDE4MDM0OTE@._V1_UY209_CR6,0,140,209_AL_.jpg',
            'https://m.media-amazon.com/images/M/MV5BOWUzNjU3YTUtOGIyMi00MDk3LTk1NDMtMzY4MDZlNDFhNTFhXkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR73,0,140,209_AL_.jpg',
            'https://m.media-amazon.com/images/M/MV5BNTk1OTUxMzIzMV5BMl5BanBnXkFtZTcwMzMxMjI0Nw@@._V1_UY209_CR6,0,140,209_AL_.jpg',
            'https://m.media-amazon.com/images/M/MV5BNzZhNDQ1YzItNTVlOC00MmVmLTkzMjEtMDFmN2Q0MDU2YmQ3XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR21,0,140,209_AL_.jpg',
            'https://m.media-amazon.com/images/M/MV5BMTU2NzkzODA3OV5BMl5BanBnXkFtZTcwMzk2MDMyOA@@._V1_UX140_CR0,0,140,209_AL_.jpg',
         

In [71]:
for name in Names:
    filename = name
    image_file = open(filename +".jpeg","wb")

for img in image_links: 
    #print(img)
    urllib.request.urlretrieve(img, filename)

image_file.close()

In [9]:
#Download the images 
for img in image_links:
    image_content = requests.get(img).content
    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join('C:\\Users\\Hrushit\\OneDrive\\Desktop\\Web Scraping\\IMDb- Bollywood Celebrities',hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path,'wb') as f:
            image.save(f, "JPEG",quality=85)
        print(f"SUCCESS - saved {img} - as {file_path}")
    except Exception as e:
        print(f"Error - could not save {img} -{e} ")

SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMjAwMjk3NDUzN15BMl5BanBnXkFtZTcwNjI4MTY0NA@@._V1_UX140_CR0,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\b930a11d3d.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BODI4NDY1NzkyM15BMl5BanBnXkFtZTgwNzM3MDM0OTE@._V1_UY209_CR13,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\93d4bca754.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMTY0MDY2MDMwOV5BMl5BanBnXkFtZTgwNDE4MDM0OTE@._V1_UY209_CR6,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\19e606debf.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BOWUzNjU3YTUtOGIyMi00MDk3LTk1NDMtMzY4MDZlNDFhNTFhXkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR73,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\606bfa414d.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV

SUCCESS - saved https://m.media-amazon.com/images/M/MV5BYjhjNWM1MmUtM2M3My00MWZmLWE5NDYtNTI5YTM3MWQwNGVhXkEyXkFqcGdeQXVyNDUzOTQ5MjY@._V1_UX140_CR0,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\5e87f020b4.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMzg0NTM0MjYxN15BMl5BanBnXkFtZTcwMzY1NTAzMQ@@._V1_UY209_CR68,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\f4a3134d58.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BNzY3ODZmNDctZDA0ZC00NGNmLTgzZmItM2JhMTI4MzM3NTU2XkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR0,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\0c22329b4f.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMjA4MjYwNzAzM15BMl5BanBnXkFtZTcwMzUyMTUxNw@@._V1_UX140_CR0,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\bb42495e6e.jpg
SUCCESS - saved https:/

SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMmY4YzA5Y2UtYmI2Yi00MzNmLWJmZjQtYTY3YWYyYzBjYWQ3XkEyXkFqcGdeQXVyOTAyMDgxODQ@._V1_UY209_CR115,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\1ef5e5da04.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMzI0OWZlOTQtNGJiNS00YzVhLWE1MGQtZGY3ZWUwYWJiOTZiXkEyXkFqcGdeQXVyMTExNDQ2MTI@._V1_UY209_CR14,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\ed04451cb3.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BMjAxNzA0MzkzNl5BMl5BanBnXkFtZTcwNjMyOTM0MQ@@._V1_UY209_CR89,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\635ef1666f.jpg
SUCCESS - saved https://m.media-amazon.com/images/M/MV5BOTFjOGIwMmUtY2FlMS00ZWJlLThhOWItY2RmY2JmYTc0YTk5XkEyXkFqcGdeQXVyMjYwMDk5NjE@._V1_UX140_CR0,0,140,209_AL_.jpg - as C:\Users\Hrushit\OneDrive\Desktop\Web Scraping\IMDb- Bollywood Celebrities\567

In [11]:
connect(db='Bollywood_Celebs_Database', host="192.168.0.106", port=27017)

meta = {"collection": "Celebs"}

celeb_name = fields.StringField(required=True)
image = fields.ImageField(thumbnail_size=(150, 150, False))
description = fields.StringField()

for celeb in bollywood_dictionary:
    name = 

Names
Images
Info
